In [2]:
import warnings

import pandas as pd

warnings.filterwarnings("ignore")

from gene_benchmark.descriptor import NCBIDescriptor
from gene_benchmark.encoder import SentenceTransformerEncoder
from gene_benchmark.tasks import EntitiesTask, load_task_definition


# How to use Granite to understand Gene
In this notebook you will learn how to use granite to create informative embeddings for gene.  
The first step will be to build the embeddings.  
We can use any gene symbol that we would like but we will use one from the 'long vs short range TF' task

In [18]:
task = load_task_definition("long vs short range TF",tasks_folder="../tasks")
gene = task.entities['symbol'][0]
print(gene)

GATA3


Now we will expand to the symbol to it's NCBI description

In [21]:
description_builder=NCBIDescriptor()
descriptions = description_builder.describe(pd.Series([gene]))
print(descriptions[0])

Gene symbol GATA3 full name GATA binding protein 3 with the summary This gene encodes a protein which belongs to the GATA family of transcription factors. The protein contains two GATA-type zinc fingers and is an important regulator of T-cell development and plays an important role in endothelial cell biology. Defects in this gene are the cause of hypoparathyroidism with sensorineural deafness and renal dysplasia. [provided by RefSeq, Nov 2009].


In [16]:
task.entities['symbol'][0]

'GATA3'

In [ ]:
granite_embedding = "ibm-granite/granite-embedding-125m-english"
granite_embedder = SentenceTransformerEncoder(granite_embedding)
NCBIDescriptor()

In [5]:
task = load_task_definition("long vs short range TF",tasks_folder="../tasks")

In [7]:
task.entities

,symbol
0,GATA3
1,SIX1
2,TEAD1
3,JUNB
4,RUNX2
...,...
169,IRF3
170,ATF1
171,STAT1
172,LHX3


In [ ]:
task = EntitiesTask(
    task="long vs short range TF",
    description_builder=NCBIDescriptor(),
    encoder=SentenceTransformerEncoder(granite_embedder),
)
_ = task.run()
task.summary()